In [ ]:
import os
import subprocess
import time
import socket
from pyngrok import ngrok
import getpass

# Change to working directory
os.chdir('/kaggle/working/')
print(os.getcwd())

def run(commands):
    """Execute shell commands and handle output"""
    for command in commands:
        with subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, bufsize=1) as sp:
            for line in sp.stdout:
                line = line.decode("utf-8", errors="replace")
                if "undefined reference" in line:
                    raise RuntimeError("Failed Processing.")
                print(line, flush=True, end="")

def wait_for_service(host, port, timeout=30):
    """Wait for a service to become available"""
    start_time = time.time()
    while time.time() - start_time < timeout:
        try:
            socket.create_connection((host, port), timeout=1)
            return True
        except socket.error:
            time.sleep(1)
    return False

def setup_environment():
    """Install required packages"""
    commands = [
        "sudo apt install lshw -y",
        "sudo apt install nginx -y",
        "pip install pyngrok ngrok --force"
    ]
    run(commands)

def install_ollama():
    """Install Ollama"""
    commands = ["curl -fsSL https://ollama.com/install.sh | sh"]
    run(commands)

def start_ollama_server():
    """Start the Ollama server with proper environment variables"""
    os.environ['OLLAMA_HOST'] = "0.0.0.0"
    os.environ['OLLAMA_ORIGINS'] = "*"

    try:
        print("Starting Ollama server...")
        server_process = subprocess.Popen(
            "/usr/local/bin/ollama serve",
            shell=True,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )
        
        # Wait for server to start
        if not wait_for_service('localhost', 11434):
            raise Exception("Ollama server failed to start")
        
        print("Ollama server started successfully")
        return server_process
        
    except Exception as e:
        print(f"Error starting Ollama server: {str(e)}")
        raise

def setup_nginx():
    """Configure and start nginx"""
    nginx_config = '''
server {
    listen 80;
    server_name *.ngrok-free.app;

    location / {
        proxy_pass http://localhost:11434;
        proxy_http_version 1.1;
        proxy_set_header Upgrade $http_upgrade;
        proxy_set_header Connection 'upgrade';
        proxy_set_header Host $host;
        proxy_cache_bypass $http_upgrade;
    }
}
'''
    
    # Write nginx configuration
    with open('/etc/nginx/conf.d/ollamasvc.conf', 'w') as f:
        f.write(nginx_config)
    
    # Stop nginx if running
    subprocess.run(["sudo", "/etc/init.d/nginx", "stop"])
    time.sleep(2)
    
    # Start nginx
    subprocess.run(["sudo", "/etc/init.d/nginx", "start"])
    time.sleep(2)
    
    # Verify nginx is running
    if not wait_for_service('localhost', 80):
        raise Exception("Nginx failed to start")

def setup_models():
    """Setup Ollama models"""
    # Write Modelfile
    with open('/kaggle/working/Modelfile3', 'w') as f:
        f.write('''FROM llama3.1
PARAMETER num_ctx 32768''')
    
    commands = [
        "ollama create -f /kaggle/working/Modelfile3 llama3.1-extra-ctx"
        #"ollama pull qwen2.5:14b"
    ]
    run(commands)
    
    # List available models
    run(["ollama list"])

def setup_ngrok():
    """Configure and start ngrok"""
    ngrok_token = getpass.getpass('Enter your NGrok access token and press enter: ')
    
    # Set ngrok auth token
    run([f"ngrok authtoken {ngrok_token}"])
    print("NGROK_TOKEN Auth Token has been set.")
    
    # Create ngrok tunnel
    listener = ngrok.connect(
        addr="localhost:80",
        metadata="Ollama API Service",
    )
    print(f"Ngrok tunnel established: {listener}")
    return listener

def main():
    try:
        # Step 1: Setup environment
        print("Setting up environment...")
        setup_environment()
        
        # Step 2: Install Ollama
        print("Installing Ollama...")
        install_ollama()
        
        # Step 3: Start Ollama server
        print("Starting Ollama server...")
        ollama_process = start_ollama_server()
        
        # Step 4: Setup nginx
        print("Setting up nginx...")
        setup_nginx()
        
        # Step 5: Setup models
        print("Setting up models...")
        setup_models()
        
        # Step 6: Setup ngrok
        print("Setting up ngrok...")
        listener = setup_ngrok()
        
        print("\nSetup completed successfully!")
        print(f"Your Ollama API is now accessible at: {listener.public_url}")
        
        # Keep the script running
        try:
            while True:
                time.sleep(1)
        except KeyboardInterrupt:
            print("Shutting down...")
            ngrok.disconnect(listener.public_url)
            ollama_process.terminate()
            subprocess.run(["sudo", "/etc/init.d/nginx", "stop"])
            
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        raise

if __name__ == "__main__":
    main()

/kaggle/working
Setting up environment...


Reading package lists...
Building dependency tree...
Reading state information...
lshw is already the newest version (02.19.git.2021.06.19.996aaad9c7-2build1).
0 upgraded, 0 newly installed, 0 to remove and 132 not upgraded.


Reading package lists...
Building dependency tree...
Reading state information...
nginx is already the newest version (1.18.0-6ubuntu14.5).
0 upgraded, 0 newly installed, 0 to remove and 132 not upgraded.
  Using cached pyngrok-7.2.3-py3-none-any.whl.metadata (8.7 kB)
  Using cached ngrok-1.4.0-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (19 kB)
  Using cached PyYAML-6.0.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
Using cached pyngrok-7.2.3-py3-none-any.whl (23 kB)
Using cached ngrok-1.4.0-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
Using cached PyYAML-6.0.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (751 kB)
  Attempting unin

/usr/lib/python3.10/subprocess.py:961: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stdout = io.open(c2pread, 'rb', bufsize)


pulling manifest ⠋ transferring model data 
pulling manifest ⠙ transferring model data 
pulling manifest ⠹ transferring model data 
pulling manifest ⠸ transferring model data 
pulling manifest ⠼ transferring model data 
pulling manifest ⠴ transferring model data 
pulling manifest 
pulling 667b0c1932bc...   0% ▕                ▏    0 B/4.9 GB                  transferring model data 
pulling manifest 
pulling 667b0c1932bc...   0% ▕                ▏    0 B/4.9 GB                  transferring model data 
pulling manifest 
pulling 667b0c1932bc...   0% ▕                ▏    0 B/4.9 GB                  transferring model data 
pulling manifest 
pulling 667b0c1932bc...   1% ▕                ▏  24 MB/4.9 GB                  transferring model data 
pulling manifest 
pulling 667b0c1932bc...   1% ▕                ▏  33 MB/4.9 GB                  transferring model data 
pulling manifest 
pulling 667b0c1932bc...   2% ▕                ▏ 101 MB/4.9 GB                  transferring model data 
pull

Enter your NGrok access token and press enter:  ········


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
NGROK_TOKEN Auth Token has been set.
Ngrok tunnel established: NgrokTunnel: "https://f6f8-104-154-160-249.ngrok-free.app" -> "http://localhost:80"

Setup completed successfully!
Your Ollama API is now accessible at: https://f6f8-104-154-160-249.ngrok-free.app
